(epistemic_propagation)=
# Propagation of epistemic uncertainty

{ref}`interval_analysis` introduces the calculation of intervals for a regirorous analysis. However, naive interval analysis has some caveats such as the dependency issue, as shown in {ref}`interval_dependency`. 

This notebook will demonstrate a wider scope of interval analysis methods that assist in acommpolishing rigirorou engineering computations by handling the caveats as well as extending to work with black box models


```{seealso}
There is an increasing awareness, among the scientific computation community, of the differentiation of aleatory and epistemic uncertainty and that different methods are needed for characterisation and propagation. See {ref}`aleatory_propagation` for the propagation of aleatory uncertainty and more realistically {ref}`mix_propagation` for a mixed situation.
```

In [22]:
import numpy as np
from pyuncertainnumber import pba, b2b
import pyuncertainnumber as pun

#### interval arithmetic

In [7]:
def f(x):
    """a universal signature that takes iterable and vectorised inputs"""

    if isinstance(x, np.ndarray):  # foo_vectorised signature
        if x.ndim == 1:
            x = x[None, :]
        return x[:, 0] ** 3 + x[:, 1] + x[:, 2]
    else:
        return x[0] ** 3 + x[1] + x[2]  # foo_iterable signature


a = pba.I(1, 5)
b = pba.I(7, 13)
c = pba.I(5, 10)   

a direct call duck-typing

In [8]:
f([a, b, c])

[13.0,148.0]

#### b2b: a bespoke function for interval propagation

`pyuncertainnumber` supports a wide spectrum of methodologies for interval propagation, intrusively and nonintrusively, including vertex method(a.k.a endpoints method), subinterval reconstitution method, Cauchy deviate method, and most generally optimistion based methods such as genetic algorithm and bayesian optimisation. Particularly, one can easily dispatch any of these methods using the function `b2b`. Here below enumerates their usage via a simple demo.

In [9]:
# endpoints
b2b(
    vars = [a, b, c],
    func = f,
    interval_strategy = 'endpoints',
)


[13.0,148.0]

In [10]:
# subinterval - direct/endpoints
b2b(
    vars = [a, b, c],
    func = f,
    interval_strategy = "subinterval",
    subinterval_style= "direct",
    n_sub=20
)

[13.0,148.0]

In [ ]:
%%capture
# optimisation through genetic algorithm
r = b2b(
    vars = [a, b, c],
    func = f,
    interval_strategy = "ga",
)

In [12]:
print(r)

[13.01861169516981,147.7571712576113]


In [13]:
%%capture
# optimisation through bayesian optimisation
rr = b2b(
    vars = [a, b, c],
    func = f,
    interval_strategy = "bo",
)

In [14]:
print(rr)

[13.0,148.0]


In [21]:
# cauchy deviate method
b2b(
    vars = [a, b, c],
    func = f,
    interval_strategy = "cauchy_deviate",
    n_sam=40_000,
)

[-11.655025046085044,100.65502504608504]

#### Propagation: a high-level API for propagation with uncertain numbers

The advantage of `Propagation` is that it automatically provides propagation capability with any types of uncertain number, whatever their underlying uncertainty representations (constructs) are. Apparently, for an analysis with all interval-value variables, it also works though seems a bit an overkill.

See xx for an engineering example.

In [23]:
a = pun.I(1, 5)
b = pun.I(7, 13)
c = pun.I(5, 10)

p = pun.Propagation(
    vars = [a, b, c],
    func = f, 
    method = 'subinterval',
)

INFO: interval propagation


In [26]:
p.run(subinterval_style="direct", n_sub=20)

UncertainNumber(essence='interval', intervals=[13.0,148.0], _construct=[13.0,148.0], nominal_value=80.5)